In [1]:
import pandas as pd
import sqlite3

dataframes needed:
- one with hospitals (Michael said there wasn't any real programatic way to do it, get a df with entity_type = 2 and then figure out the real list by looking at names)
- use hospital df to get list of referees

In [32]:
# building table for hospitals
# cbsa = 34980, either hardcode or use a cte to get the value where usps_zip_pref_city = "NASHVILLE" and usps_zip_pref_state = "TN" (filter based on referee cbsa)
query = """
SELECT npi,
    entity_type_code AS entity_type,
    provider_organization_name_legal_business_name AS provider_org_name,
    provider_first_line_business_practice_location_address AS provider_address_1,
    provider_second_line_business_practice_location_address AS provider_address_2,
    provider_business_practice_location_address_city_name AS provider_city,
    provider_business_practice_location_address_state_name AS provider_state,
    provider_business_practice_location_address_postal_code AS provider_post_code,
    extracted_taxonomy_code AS tax_code,
    grouping AS code_group,
    classification AS code_classification,
    display_name AS code_display_name
FROM npi AS n
LEFT JOIN class_cross AS c
    ON n.extracted_taxonomy_code = c.code
LEFT JOIN cbsa_cross cb
    ON n.provider_business_practice_location_address_postal_code = cb.zip
WHERE entity_type_code = 2
    AND cbsa = 34980;
"""

with sqlite3.connect('../data/provider.sqlite') as db: 
    hosp_sqlite = pd.read_sql(query, db)

hosp_sqlite.head()

,npi,entity_type,provider_org_name,provider_address_1,provider_address_2,provider_city,provider_state,provider_post_code,tax_code,code_group,code_classification,code_display_name
0,1881697092,2,"RHS, INC.",1330 TROTWOOD AVE,None,COLUMBIA,TN,38401,332B00000X,Suppliers,Durable Medical Equipment & Medical Supplies,Durable Medical Equipment & Medical Supplies
1,1326041534,2,MURFREESBORO CONVALESCENT SERVICE,4428 LASCASSAS PIKE,None,MURFREESBORO,TN,37130,341600000X,Transportation Services,Ambulance,Ambulance
2,1063415834,2,ADVANCED DIABETIC SERVICES,151 HERITAGE PARK DR,STE 403,MURFREESBORO,TN,37129,332B00000X,Suppliers,Durable Medical Equipment & Medical Supplies,Durable Medical Equipment & Medical Supplies
3,1538162334,2,TENNESSEE IMAGING ALLIANCE LLC,2628 N MOUNT JULIET RD,None,MOUNT JULIET,TN,37122,261QR0208X,Ambulatory Health Care Facilities,Clinic/Center,Mobile Radiology Clinic/Center
4,1245233220,2,"NORTHRIDGE SURGERY CENTER, LP",601 SAUNDERSVILLE RD,None,MADISON,TN,37075,261QA1903X,Ambulatory Health Care Facilities,Clinic/Center,Ambulatory Surgical Clinic/Center


In [34]:
hosp_sqlite['code_group'].value_counts()

Allopathic & Osteopathic Physicians                                             2002
Ambulatory Health Care Facilities                                               1408
Suppliers                                                                       1354
Agencies                                                                         888
Dental Providers                                                                 662
Behavioral Health & Social Service Providers                                     440
Nursing & Custodial Care Facilities                                              332
Chiropractic Providers                                                           332
Physician Assistants & Advanced Practice Nursing Providers                       329
Transportation Services                                                          327
Other Service Providers                                                          317
Respiratory, Developmental, Rehabilitative and Restorative Servic

In [58]:
# list of "hospital" NPIs to build profile
# all located in Nashville CBSA
hosp_npi = tuple(hosp_sqlite[hosp_sqlite['code_group'].isin(['Hospitals', 'Hospital Units', 'Ambulatory Health Care Facilities'])]['npi'].tolist())

In [60]:
# get dataframe of hospitals in Nashville CBSA using previously generated NPI list
query = f"""
SELECT npi,
    entity_type_code AS entity_type,
    provider_organization_name_legal_business_name AS provider_org_name,
    provider_first_line_business_practice_location_address AS provider_address_1,
    provider_second_line_business_practice_location_address AS provider_address_2,
    provider_business_practice_location_address_city_name AS provider_city,
    provider_business_practice_location_address_state_name AS provider_state,
    provider_business_practice_location_address_postal_code AS provider_post_code,
    extracted_taxonomy_code AS tax_code,
    grouping AS code_group,
    classification AS code_classification,
    display_name AS code_display_name
FROM npi AS n
LEFT JOIN class_cross AS c
    ON n.extracted_taxonomy_code = c.code
LEFT JOIN cbsa_cross cb
    ON n.provider_business_practice_location_address_postal_code = cb.zip
WHERE npi IN {hosp_npi}
    AND cbsa = 34980;
"""

with sqlite3.connect('../data/provider.sqlite') as db: 
    hosp = pd.read_sql(query, db)

hosp.head()

,npi,entity_type,provider_org_name,provider_address_1,provider_address_2,provider_city,provider_state,provider_post_code,tax_code,code_group,code_classification,code_display_name
0,1740720275,2,CHS TENNESSEE MEDICAL PC,4538 GUTHRIE HWY,None,CLARKSVILLE,TN,37010,261QM1300X,Ambulatory Health Care Facilities,Clinic/Center,Multi-Specialty Clinic/Center
1,1003408485,2,"NASHVILLE DENTURES & IMPLANTS, PLLC",504 COLLINS PARK DR,None,ANTIOCH,TN,37013,261QD0000X,Ambulatory Health Care Facilities,Clinic/Center,Dental Clinic/Center
2,1023498250,2,ANTIOCH DENTAL,616 BELL RD,None,ANTIOCH,TN,37013,261QD0000X,Ambulatory Health Care Facilities,Clinic/Center,Dental Clinic/Center
3,1316305089,2,"MIDDLE TENNESSEE IMAGING, LLC",3754 MURFREESBORO PIKE STE 102,None,ANTIOCH,TN,37013,261QR0200X,Ambulatory Health Care Facilities,Clinic/Center,Radiology Clinic/Center
4,1316529118,2,"HEALING OASIS THERAPY, LLC",3538 MURFREESBORO PIKE STE 301,None,ANTIOCH,TN,37013,261QM0850X,Ambulatory Health Care Facilities,Clinic/Center,Adult Mental Health Clinic/Center
